In [ ]:
import argparse, sys, json
from reporte_python.bdp_report_coach import generate_report_coach
from reporte_python.helpers import _preprocess_csv_for_coach, _resolve_columns
from reporte_python.bdp_schema import DEFAULT_COLUMNS
import pandas as pd
import numpy as np

input_csv ="data/bdp_sample.csv"
targets = "animo,claridad,estres,activacion".split(",")
output ="./outputs"
target_email = "test@gmail.com"
lookback_days = 6
row_idx = -1
target_day = "26-08-2025"
start_date = None
end_date  = None
tag_filter = None


In [2]:
# Preprocesa: normaliza hora 'a. m./p. m.', garantiza 'fecha'/'hora', filtra por correo
pre_csv = _preprocess_csv_for_coach(input_csv, target_email)

In [3]:
df = pd.read_csv(pre_csv, dtype={"hora": "string"})
df.columns = [str(c).strip() for c in df.columns]
# mapea a nombres esperados
df = _resolve_columns(df, DEFAULT_COLUMNS)
if "fecha" in df.columns:
    df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce", dayfirst=True)

if start_date:
    sd = pd.to_datetime(start_date, errors="coerce", dayfirst=True)
    df = df[df["fecha"] >= sd]
if end_date:
    ed = pd.to_datetime(end_date, errors="coerce", dayfirst=True)
    df = df[df["fecha"] <= ed]

if tag_filter and "tags" in df.columns:
    df = df[df["tags"].astype(str).str.contains(tag_filter, case=False, na=False)]

In [4]:
from lab.bdp_pipeline import load_csv_or_template, build_and_evaluate
import os
results = build_and_evaluate(df, export_dir="./outputs")

# Guardo el dataframe enriquecido por si se quiere inspeccionar
enr_path = os.path.join("./outputs", "bdp2_out.csv")
results["df"].to_csv(enr_path, index=False, encoding="utf-8")
print(f"[OK] Pipeline completo. Archivos en: {os.path.abspath("./outputs")}")

[AVISO] Muy pocos datos para estres: 21 filas. Se entrenará igual, pero la validación puede ser débil.
[AVISO] Muy pocos datos para animo: 21 filas. Se entrenará igual, pero la validación puede ser débil.
[AVISO] Muy pocos datos para claridad: 21 filas. Se entrenará igual, pero la validación puede ser débil.
[AVISO] Muy pocos datos para activacion: 21 filas. Se entrenará igual, pero la validación puede ser débil.

================= MODELOS & POLÍTICA DE PESOS =================
[estres] best=RandomForest  MAE_model=0.999  MAE_EMA=1.214  Δ%vsEMA=0.178
      Pesos sugeridos → model=0.43, ema14=0.57
[animo] best=RandomForest  MAE_model=0.509  MAE_EMA=0.428  Δ%vsEMA=-0.19
      Pesos sugeridos → model=0.3, ema14=0.7
[claridad] best=ElasticNetCV  MAE_model=0.379  MAE_EMA=0.745  Δ%vsEMA=0.492
      Pesos sugeridos → model=0.5, ema14=0.5
[activacion] best=RandomForest  MAE_model=0.671  MAE_EMA=0.616  Δ%vsEMA=-0.09
      Pesos sugeridos → model=0.3, ema14=0.7

[OK] Pipeline completo. Archivos e

In [7]:
from bdp_calcs.relations import wellbeing_index

# Estresores y su impacto diario

In [8]:
cols = ['bienestar_diario',"fecha",'animo', 'activacion', 'conexion', 'movimiento', 'irritabilidad',
        'claridad', 'autocuidado','alimentacion', 'ansiedad','estres', 'sueno_calidad', 'despertares_nocturnos',
        'dolor_fisico', 'mov_intensidad', 
        'meditacion_min', 'agua_litros', 'cafe_cucharaditas', 'alcohol_ud','notas'
       ]

In [9]:
targets = ['animo', 'activacion', 
        'claridad','estres', 'sueno_calidad']
wb = wellbeing_index(df, targets=targets)
wb_trend = "estable"
if len(wb.dropna()) >= 2:
    dwb = wb.diff().iloc[-1]
    if pd.notna(dwb):
        if dwb > 0.4:
            wb_trend = "al alza"
        elif dwb < -0.4:
            wb_trend = "a la baja"
        else:
            wb_trend = "estable"

In [10]:
df["bienestar_diario"]= wb.astype(np.float64)
subset = df[cols]
subset = subset.fillna(0)
stressout = subset[subset["fecha"] == "2025-08-24"]